# Example attacks against Large Language Models with TextAttack

In this notebook, we will show you how to use Textattack, a library to test the robustness of LLMs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zangobot/teaching_material/blob/HEAD/06-LLMSecurity.ipynb)

In [1]:
!pip install textattack tensorflow tensorflow_hub

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import transformers

An attack in TextAttack consists of four parts:

- **Goal function**: determines if the attack is successful or not: evaluates how well a perturbed attacked_text object is achieving a specified goal. A common goal function is **targeted**/**untargeted classification**, where the attack tries to perturb an input to change its classification.

- **Transformation**: takes a text input and transforms it, for example by replacing words or phrases with similar ones, while trying not to change the meaning. Paraphrase and synonym substitution are two broad classes of transformations.

- **Constraints**: determine whether or not a given transformation is valid. Transformations don't perfectly preserve syntax or semantics, so additional constraints can increase the probability that these qualities are preserved from the source to adversarial example. There are many types of constraints: overlap constraints that measure edit distance, syntactical  constraints check part-of-speech and grammar errors, and semantic constraints like language models and sentence encoders.

- **Search method**: explores the space of possible transformations within the defined constraints and attempt to find a successful perturbation which satisfies the goal function. Some examples are greedy search with word importance ranking, beam search, brute force, and genetic algorithm.


In this way, attacking an NLP model can be framed as a combinatorial search problem. The attacker must search within all potential transformations to find a sequence of transformations that generate a successful adversarial example.

TextAttack provides a set of **Attack Recipes** that assemble attacks from the literature from these four components. We can exploit an attack recipe or build an attack from scratch.


# Example 1: Attack Recipe

In [3]:
from textattack.datasets import HuggingFaceDataset

# load the dataset from HuggingFace
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test")

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape s

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

textattack: Loading datasets dataset rotten_tomatoes, split test.


In [4]:
# print some examples
print(dataset[0])
print(dataset[1])
print(dataset[2])
print(dataset[3])
print(dataset[4])

(OrderedDict({'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .'}), 1)
(OrderedDict({'text': 'consistently clever and suspenseful .'}), 1)
(OrderedDict({'text': 'it\'s like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .'}), 1)
(OrderedDict({'text': 'the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .'}), 1)
(OrderedDict({'text': 'red dragon " never cuts corners .'}), 1)


In [5]:
from textattack.models.wrappers import HuggingFaceModelWrapper

# load a pre-trained BERT-based model for sequence classification from the HuggingFace model hub
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/distilbert-base-uncased-rotten-tomatoes"
)

# load the corresponding tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/distilbert-base-uncased-rotten-tomatoes"
)

# TextAttack requires both the model and the tokenizer to be wrapped by a ModelWrapper class that implements the forward pass operation given a list of input texts
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
print("\nMODEL: ", model)
print("\nTOKENIZER: ", tokenizer)
print("\nWRAPPER: ", model_wrapper)


MODEL:  DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplac

Let's use TextFooler, which operates at word-level.

The main idea behind **TextFooler** is to generate adversarial examples by iteratively replacing or modifying words in the input text to maximize the model's prediction error while minimizing the perturbation's perceptibility to human readers. The framework utilizes semantic and syntactic similarity metrics to guide the perturbation process, ensuring that the modified text remains grammatically and semantically coherent.

In [7]:
from textattack.attack_recipes import TextFoolerJin2019

# load the attack
attack = TextFoolerJin2019.build(model_wrapper)

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.

  0%|          | 0.00/481M [00:00<?, ?B/s]
  0%|          | 69.6k/481M [00:00<14:05, 569kB/s]
  0%|          | 296k/481M [00:00<06:02, 1.33MB/s]
  0%|          | 1.22M/481M [00:00<01:55, 4.16MB/s]
  1%|          | 4.76M/481M [00:00<00:31, 15.0MB/s]
  2%|▏         | 7.55M/481M [00:00<00:24, 19.3MB/s]
  2%|▏         | 10.5M/481M [00:00<00:20, 22.5MB/s]
  3%|▎         | 12.9M/481M [00:00<00:20, 22.9MB/s]
  3%|▎         | 15.3M/481M [00:00<00:20, 22.3MB/s]
  4%|▎         | 17.8M/481M [00:00<00:19, 23.3MB/s]
  4%|▍         | 21.6M/481M [00:01<00:16, 27.7MB/s]
  5%|▌         | 26.1M/481M [00:01<00:13, 32.6MB/s]
  6%|▌         | 29.9M/481M [00:01<00:13, 34.2MB/s]
  7%|▋         | 34.8M/481M [00:01<00:12, 36.4MB/s]
  8%|▊         | 40.1M/481M [00:01<00:10, 41.0MB/s]
  9%|▉         | 44.2M/481M [00:01<00:10, 40.3MB/s]
 10%|█         | 48.3M/481M [00:01<00:10, 40.0MB/s]
 11%|█         | 53.6M/481M [00:01<00:

In [8]:
print("\nATTACK: ", attack)


ATTACK:  Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis

By printing the attack we can check the definition of the four components: search method, goal function, transformation, constraints.

Let's run the attack.

In [9]:
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples = 10)

attacker = Attacker(attack, dataset, attack_args)

TextAttack attacks iterate through a dataset and, for each correctly predicted sample, search for an adversarial perturbation. If an example is incorrectly predicted to begin with, it is not attacked (Skipped), since the input already fools the model.

In [12]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

attack_results = attacker.attack_dataset()

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


  0%|          | 0/10 [01:13<?, ?it/s]

 10%|█         | 1/10 [00:37<05:37, 37.51s/it]

--------------------------------------------- Result 1 ---------------------------------------------



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:38<05:44, 38.32s/it]

...

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (92%)]] --> [[Negative (70%)]]

throws in enough clever and [[unexpected]] [[twists]] to make the formula feel fresh .

throws in enough clever and [[unwanted]] [[tendrils]] to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (68%)]] --> [[Negative (60%)]]

weighty and ponderous but [[every]] bit as filling as the treat of the title .

weighty and ponderous but [[any]] bit as filling as the treat of the title .





[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9:  90%|█████████ | 9/10 [00:42<00:04,  4.73s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (94%)]] --> [[Negative (73%)]]

a [[real]] audience-pleaser that [[will]] [[strike]] a [[chord]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

a [[hardheaded]] audience-pleaser that [[going]] [[slugged]] a [[chords]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .





[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9: 100%|██████████| 10/10 [00:43<00:00,  4.32s/it]
[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10: 100%|██████████| 10/10 [00:43<00:00,  4.32s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (96%)]] --> [[Negative (55%)]]

generates an [[enormous]] feeling of empathy for its characters .

generates an [[dreaded]] feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 70.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 16.54% |
| Average num. words per input: | 15.4   |
| Avg num queries:              | 52.86  |
+-------------------------------+--------+


To visualize the results, we are logging AttackResult objects using a CSVLogger. This logger stores all attack results in a dataframe, which we can easily access and display. Since we set color_method to 'html', the attack results will display their differences, in color, in HTML. Using IPython utilities and pandas

In [13]:
import pandas as pd
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult

# pd.options.display.max_colwidth = (
#     480  # increase colum width so we can actually read the examples
# )

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,"lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .","lovingly photographed in the manner of a golden book sprung to life , stuart little 2 runs sweetness largely without stickiness ."
1,consistently clever and suspenseful .,consistently malin and nightmarish .
2,"the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .","the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with stupendous jurisdictional ."
3,throws in enough clever and unexpected twists to make the formula feel fresh .,throws in enough clever and unwanted tendrils to make the formula feel fresh .
4,weighty and ponderous but every bit as filling as the treat of the title .,weighty and ponderous but any bit as filling as the treat of the title .
5,"a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .","a hardheaded audience-pleaser that going slugged a chords with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office ."
6,generates an enormous feeling of empathy for its characters .,generates an dreaded feeling of empathy for its characters .


# Example 2: Attack from scratch


In [14]:
# Load the dataset
from textattack.datasets import HuggingFaceDataset

dataset = HuggingFaceDataset("ag_news", None, "test")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

textattack: Loading datasets dataset ag_news, split test.


In [15]:
# Load the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained(  # loads a pre-trained BERT-based model for sequence classification from the HuggingFace model hub
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained( # loads the corresponding tokenizer for the BERT model
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### Banana word swap
We're going to try a very simple transformation: one that replaces any given word with the word 'banana'. In TextAttack, there's an abstract WordSwap class that handles the heavy lifting of breaking sentences into words and avoiding replacement of stopwords. We can extend WordSwap and implement a single method, _get_replacement_words, to indicate to replace each word with 'banana'. 🍌

In [16]:
from textattack.transformations import WordSwap

class BananaWordSwap(WordSwap):
    """Transforms an input by replacing any word with 'banana'."""

    def _get_replacement_words(self, word):
        """Returns a list with one item, since `_get_replacement_words` is intended to return a list of candidate replacement words."""
        return ["banana"]

In [17]:
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import (
    RepeatModification,   # constraint disallowing the modification of words which have already been modified
    StopwordModification, # constraint disallowing the modification of stopwords
)
from textattack import Attack

# Create the goal function using the model
goal_function = UntargetedClassification(model_wrapper)

# Use Banana word swap class as the attack transformation
transformation = BananaWordSwap()

# Avoid modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]

# Use the Greedy search method
search_method = GreedySearch()

# Create the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [18]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [19]:
from tqdm import tqdm
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples=10)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




 10%|█         | 1/10 [00:02<00:25,  2.79s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:02<00:25,  2.80s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (100%)]] --> [[World (89%)]]

Fears for T N [[pension]] after [[talks]] [[Unions]] representing [[workers]] at Turner   Newall say they are '[[disappointed']] after talks with stricken parent firm Federal [[Mogul]].

Fears for T N [[banana]] after [[banana]] [[banana]] representing [[banana]] at Turner   Newall say they are '[[banana]] after talks with stricken parent firm Federal [[banana]].





[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  20%|██        | 2/10 [00:22<01:28, 11.07s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:22<01:28, 11.08s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Sci/tech (100%)]] --> [[World (64%)]]

The Race is On: Second Private [[Team]] Sets Launch [[Date]] for [[Human]] [[Spaceflight]] ([[SPACE]].[[com]]) [[SPACE]].[[com]] - [[TORONTO]], [[Canada]] -- [[A]] [[second]]\[[team]] of rocketeers [[competing]] for the  #36;10 million Ansari X [[Prize]], a [[contest]] for\[[privately]] funded [[suborbital]] [[space]] [[flight]], has officially [[announced]] the first\[[launch]] date for its [[manned]] rocket.

The Race is On: Second Private [[banana]] Sets Launch [[banana]] for [[banana]] [[banana]] ([[banana]].[[banana]]) [[banana]].[[banana]] - [[banana]], [[banana]] -- [[banana]] [[banana]]\[[banana]] of rocketeers [[banana]] for the  #36;10 million Ansari X [[banana]], a [[banana]] for\[[banana]] funded [[banana]] [[banana]] [[banana]], has officially [[banana]] the first\[[banana]] date for its [[banana]] rocket.




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

FOAFKey: FOAF, PGP, Key Distribution, and Bloom Filters \\FOAF/LOAF  and bloom filters have a lot of interesting properties for social\network and whitelist distribution.\\I think we can go one level higher though and include GPG/OpenPGP key\fingerpring distribution in the FOAF file for simple web-of-trust based key\distribution.\\What if we used FOAF and included the PGP key fingerprint(s) for identities?\This could mean a lot.  You include the PGP key fingerprints within the FOAF\file of your direct friends and then include a bloom filter of the PGP key\fingerprints of your entire whitelist (the source FOAF file would of course need\to be encrypted ).\\Your whitelist would be populated from the social network as your client\discovered new identit ...\\





[Succeeded / Failed / Skipped / Total] 6 / 2 / 0 / 8:  90%|█████████ | 9/10 [04:55<00:32, 32.88s/it]
[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [04:55<00:32, 32.88s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Sci/tech (98%)]] --> [[World (100%)]]

[[E-mail]] scam targets police chief Wiltshire Police warns about "[[phishing]]" after its fraud squad chief was targeted.

[[banana]] scam targets police chief Wiltshire Police warns about "[[banana]]" after its fraud squad chief was targeted.





[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9: 100%|██████████| 10/10 [04:56<00:00, 29.67s/it]
[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [04:56<00:00, 29.67s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Sci/tech (98%)]] --> [[World (77%)]]

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[card]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.

Card fraud unit nets 36,000 cards In its first two years, the UK's dedicated [[banana]] fraud unit, has recovered 36,000 stolen cards and 171 arrests - and estimates it saved 65m.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 18.71% |
| Average num. words per input: | 63.0   |
| Avg num queries:     

In [20]:
import pandas as pd
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult

# pd.options.display.max_colwidth = (
#     480  # increase colum width so we can actually read the examples
# )

logger = CSVLogger(color_method="html")

for result in attack_results:
    if isinstance(result, SuccessfulAttackResult):
        logger.log_attack_result(result)

from IPython.core.display import display, HTML

results = pd.DataFrame.from_records(logger.row_list)
display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv
textattack: CSVLogger exiting without calling flush().


,original_text,perturbed_text
0,Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Fears for T N banana after banana banana representing banana at Turner Newall say they are 'banana after talks with stricken parent firm Federal banana.
1,"The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.","The Race is On: Second Private banana Sets Launch banana for banana banana (banana.banana) banana.banana - banana, banana -- banana banana\banana of rocketeers banana for the #36;10 million Ansari X banana, a banana for\banana funded banana banana banana, has officially banana the first\banana date for its banana rocket."
2,"Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.","Ky. Company Wins Grant to banana banana (banana) banana - banana company banana by a banana banana at the banana of Louisville won a grant to develop a method of producing better banana, which are short chains of banana banana, the building blocks of banana."
3,"Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.","banana Unit Helps banana Wildfires (AP) banana - It's barely dawn when Mike Fitzpatrick banana his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar."
4,"Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.","Calif. Aims to Limit Farm-Related banana (AP) AP - Southern California's banana agency went after banana of the banana variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure."
5,"Open Letter Against British Copyright Indoctrination in Schools The British Department for Education and Skills (DfES) recently launched a ""Music Manifesto"" campaign, with the ostensible intention of educating the next generation of British musicians. Unfortunately, they also teamed up with the music industry (EMI, and various artists) to make this popular. EMI has apparently negotiated their end well, so that children in our schools will now be indoctrinated about the illegality of downloading music.The ignorance and audacity of this got to me a little, so I wrote an open letter to the DfES about it. Unfortunately, it's pedantic, as I suppose you have to be when writing to goverment representatives. But I hope you find it useful, and perhaps feel inspired to do something similar, if or when the same thing has happened in your area.","Open banana Against banana banana Indoctrination in Schools The banana Department for Education and Skills (DfES) banana banana a ""banana banana"" campaign, with the ostensible banana of banana the banana banana of banana banana. banana, they also teamed up with the banana industry (banana, and banana banana) to make this popular. banana has banana banana their end well, so that banana in our schools will now be indoctrinated about the illegality of banana music.The ignorance and audacity of this got to me a little, so I wrote an op

Looks like some examples needed only a couple "banana"s, while others needed up to 17 "banana" substitutions to change the class score.